In [12]:
"""
The basis for a framework of automatic differentiation such as PyTorch.

What we want is to:
1. Build a forward pass manually, passing tensors / variable through blocks
2. Make sure each variable remember which block uses it (to get its gradients)
"""

import abc
import math
import numpy as np

In [23]:
class Variable:
    def __init__(self, value):
        self.value = value
        self.gradient = None
        self.gradient_fcts = []
    
    def compute_gradient(self):
        self.gradient = sum(fct() for fct in self.gradient_fcts) if self.gradient_fcts else 1
        return self.gradient
    
    def backward(self):
        # TODO - in need for a graph to do a kind of topological sort?
        pass
    
    def __repr__(self):
        return str({
            "value": self.value,
            "gradient_fcts": self.gradient_fcts
        })

    
class AddFct:
    def __init__(self, destination):
        self.destination = destination
    
    def __call__(self):
        return self.destination.gradient
    
    def __repr__(self):
        return str({
            "destination": self.destination,
            "function": "Add"
        })
    
    
def add(v1: Variable, v2: Variable):
    total = Variable(v1.value + v2.value)
    v1.gradient_fcts.append(AddFct(total))
    v2.gradient_fcts.append(AddFct(total))
    return total
    
x1 = Variable(1)
x2 = Variable(2)
x3 = Variable(3)
y1 = add(x1, x2)
y2 = add(x2, x3)
z = add(y1, y2)

print(x1)
print(x2)
print(x3)

print(z.compute_gradient())
print(y1.compute_gradient())
print(y2.compute_gradient())
print(x1.compute_gradient())
print(x2.compute_gradient())
print(x3.compute_gradient())

{'value': 1, 'gradient_fcts': [{'destination': {'value': 3, 'gradient_fcts': [{'destination': {'value': 8, 'gradient_fcts': []}, 'function': 'Add'}]}, 'function': 'Add'}]}
{'value': 2, 'gradient_fcts': [{'destination': {'value': 3, 'gradient_fcts': [{'destination': {'value': 8, 'gradient_fcts': []}, 'function': 'Add'}]}, 'function': 'Add'}, {'destination': {'value': 5, 'gradient_fcts': [{'destination': {'value': 8, 'gradient_fcts': []}, 'function': 'Add'}]}, 'function': 'Add'}]}
{'value': 3, 'gradient_fcts': [{'destination': {'value': 5, 'gradient_fcts': [{'destination': {'value': 8, 'gradient_fcts': []}, 'function': 'Add'}]}, 'function': 'Add'}]}
1
1
1
1
2
1


In [10]:
import torch
x = torch.zeros((1, 1), requires_grad=True)
print(x)
y = torch.exp(x)
print(y)

tensor([[0.]], requires_grad=True)
tensor([[1.]], grad_fn=<ExpBackward>)
